# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [7]:
import pandas as pd

df1 = pd.read_csv('cleaned_customer_data.csv')

def clean_customer_data(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

    if 'st' in df.columns:
        df.rename(columns={'st': 'state'}, inplace=True)

    df['gender'] = df['gender'].str.strip().str.lower().replace({
        'f': 'F', 'female': 'F', 'femal': 'F',
        'm': 'M', 'male': 'M'
    })

    df['state'] = df['state'].replace({
        'AZ': 'Arizona',
        'Cali': 'California',
        'WA': 'Washington'
    })

    df['education'] = df['education'].replace({'Bachelors': 'Bachelor'})

    df['vehicle_class'] = df['vehicle_class'].replace({
        'Luxury SUV': 'Luxury',
        'Luxury Car': 'Luxury',
        'Sports Car': 'Luxury'
    })

    df['customer_lifetime_value'] = df['customer_lifetime_value'].astype(str).str.replace('%', '', regex=False)
    df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

    def extract_complaint_number(val):
        if isinstance(val, str) and '/' in val:
            parts = val.split('/')
            if len(parts) >= 2 and parts[1].isdigit():
                return int(parts[1])
        elif pd.isna(val):
            return val
        else:
            try:
                return int(val)
            except:
                return None

    df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(extract_complaint_number)
    df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce')

    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())

    categorical_cols = df.select_dtypes(include='object').columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    for col in numerical_cols:
        df[col] = df[col].astype(int)

    df = df.drop_duplicates().reset_index(drop=True)

    return df

url2 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
url3 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"

df2 = pd.read_csv(url2)
df3 = pd.read_csv(url3)

df2_cleaned = clean_customer_data(df2)
df3_cleaned = clean_customer_data(df3)

final_df = pd.concat([df1, df2_cleaned, df3_cleaned], ignore_index=True)

final_df.to_csv('final_combined_customer_data.csv', index=False)

print("✅ Final cleaned and combined dataset saved as 'final_combined_customer_data.csv'")
print(f"Total rows in final dataset: {final_df.shape[0]}")
                  


✅ Final cleaned and combined dataset saved as 'final_combined_customer_data.csv'
Total rows in final dataset: 9138


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.
2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [ ]:
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

print("🔍 Dataset shape:", df.shape)
print("\n🧾 First few rows:")
print(df.head())

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print("\n🧠 Data types:")
print(df.dtypes)

print("\n🕳️ Null values per column:")
print(df.isnull().sum())

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include='object').columns

print("\n📊 Numerical columns:", list(numerical_cols))
print("🔤 Categorical columns:", list(categorical_cols))

print("\n📈 Summary statistics (numerical):")
print(df[numerical_cols].describe())

print("\n🔍 Unique values per categorical column:")
for col in categorical_cols:
    print(f"- {col}: {df[col].nunique()} unique values")

🔍 Dataset shape: (10910, 27)

🧾 First few rows:
   unnamed:_0 customer       state  customer_lifetime_value response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon              9025.067525       No   

   coverage education effective_to_date employmentstatus gender  ...  \
0     Basic   College        2011-02-18         Employed      M  ...   
1     Basic   College        2011-01-18       Unemployed      F  ...   
2     Basic  Bachelor        2011-02-10         Employed      M  ...   
3  Extended   College        2011-01-11         Employed      M  ...   
4   Premium  Bachelor        2011-01-17    Medical Leave      F  ...   

   number_of_policies     policy_type        policy  renew_offer_type  \
0            

In [9]:
revenue_by_channel = pd.pivot_table(
    df,
    index='sales_channel',
    values='total_claim_amount',
    aggfunc='sum'
).round(2).sort_values(by='total_claim_amount', ascending=False)

print("📊 Total Revenue by Sales Channel:")
print(revenue_by_channel)

📊 Total Revenue by Sales Channel:
               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


In [10]:
clv_by_gender_edu = pd.pivot_table(
    df,
    index='gender',
    columns='education',
    values='customer_lifetime_value',
    aggfunc='mean'
).round(2)

print("\n🎓 Average CLV by Gender and Education:")
print(clv_by_gender_edu)


🎓 Average CLV by Gender and Education:
education  Bachelor  College   Doctor  High School or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here